 # MySQL-to-PostgreSQL data migration

Purpose of the Script :
==================
This script is designed to automate the migration of tables from a MySQL database to a PostgreSQL database while ensuring data integrity, efficiency, and minimal downtime. It is particularly useful for database migrations, archiving, or cloud transitions.

Key Objectives:
==========
Seamless Data Transfer:

Extract table schemas from MySQL and create equivalent PostgreSQL tables.
Transfer data in batches to prevent memory overload.

Schema Compatibility Handling:

Map MySQL data types to their PostgreSQL counterparts.
Preserve constraints such as NOT NULL.

Data Consistency & Integrity:

Skip already migrated rows to prevent duplication.
Handle NULL values appropriately for PostgreSQL.
Use transactions to roll back on failures.

Performance Optimization:

Process data in batches instead of migrating all at once.
Use threading to keep the script running efficiently.

Error Handling & Logging:

Capture and display errors for debugging.
Log migration progress, including skipped rows.

--This script is useful for database administrators, data engineers, and IT teams who need a reliable and automated solution for MySQL-to-PostgreSQL data migration

In [ ]:
import pymysql
import psycopg2
import sys
import time
import threading

def connect_mysql(mysql_params):
    try:
        conn = pymysql.connect(
            host=mysql_params['host'],
            user=mysql_params['username'],
            password=mysql_params['password'],
            database=mysql_params['database'],
            port=int(mysql_params['port'])
        )
        return conn
    except Exception as e:
        print(f"Error connecting to MySQL database: {e}")
        sys.exit(1)

def connect_postgresql(pg_params):
    try:
        conn = psycopg2.connect(
            host=pg_params['host'],
            port=pg_params['port'],
            database=pg_params['database'],
            user=pg_params['username'],
            password=pg_params['password']
        )
        return conn
    except Exception as e:
        print(f"Error connecting to PostgreSQL database: {e}")
        sys.exit(1)

def map_data_types(mysql_type, size=None):
    type_mapping = {
        'CHAR': f'CHAR({size})' if size else 'CHAR',
        'VARCHAR': f'VARCHAR({size})' if size else 'VARCHAR',
        'TEXT': 'TEXT',
        'INT': 'INTEGER',
        'BIGINT': 'BIGINT',
        'SMALLINT': 'SMALLINT',
        'DECIMAL': 'NUMERIC',
        'NUMERIC': 'NUMERIC',
        'FLOAT': 'DOUBLE PRECISION',
        'DOUBLE': 'DOUBLE PRECISION',
        'DATE': 'DATE',
        'DATETIME': 'TIMESTAMP',
        'TIMESTAMP': 'TIMESTAMP',
        'TINYINT': 'BOOLEAN',
    }
    return type_mapping.get(mysql_type.upper(), 'TEXT')

def get_table_schema(mysql_conn, table_name):
    try:
        cursor = mysql_conn.cursor()
        cursor.execute(f"DESCRIBE {table_name};")
        schema = []
        for column in cursor.fetchall():
            column_name = column[0]
            data_type = column[1]
            nullable = column[2] == 'YES'
            size = None
            if '(' in data_type:
                data_type, size = data_type.split('(')
                size = int(size[:-1])
            schema.append((column_name, data_type, nullable, size))
        return schema
    except Exception as e:
        print(f"Error fetching schema for table {table_name}: {e}")
        return []

def create_table_postgresql(pg_conn, source_table_name, schema, target_schema='ind_amr_nelson_syteline'):
    target_table_name = f"{target_schema}.{source_table_name}_mona1"
    cursor = pg_conn.cursor()
    columns = []
    for column in schema:
        column_name, data_type, nullable, size = column
        pg_data_type = map_data_types(data_type, size)
        nullability = "NOT NULL" if not nullable else ""
        columns.append(f"{column_name} {pg_data_type} {nullability}")
    columns_str = ", ".join(columns)
    create_table_query = f"CREATE TABLE IF NOT EXISTS {target_table_name} ({columns_str});"
    cursor.execute(create_table_query)
    pg_conn.commit()
    cursor.close()
    return target_table_name

def migrate_data(mysql_conn, pg_conn, table_name):
    print(f"Starting migration for table {table_name}...")
    schema = get_table_schema(mysql_conn, table_name)
    if not schema:
        print(f"No schema found for table {table_name}. Skipping.")
        return
    target_table_name = create_table_postgresql(pg_conn, table_name, schema, 'ind_amr_nelson_syteline')
    pg_cursor = pg_conn.cursor()
    mysql_cursor = mysql_conn.cursor()
    pg_cursor.execute(f"SELECT COUNT(*) FROM {target_table_name};")
    rows_already_inserted = pg_cursor.fetchone()[0]
    print(f"Rows already in target table: {rows_already_inserted}")
    mysql_cursor.execute(f"SELECT * FROM {table_name};")
    batch_size = 10
    rows_skipped = 0
    total_rows_processed = 0
    columns = [desc[0] for desc in mysql_cursor.description]
    while True:
        rows = mysql_cursor.fetchmany(batch_size)
        if not rows:
            break
        total_rows_processed += len(rows)
        if rows_skipped + len(rows) <= rows_already_inserted:
            rows_skipped += len(rows)
            print(f"Skipped {len(rows)} rows, total skipped: {rows_skipped}")
            continue
        elif rows_skipped < rows_already_inserted:
            rows_to_skip = rows_already_inserted - rows_skipped
            rows = rows[rows_to_skip:]
            rows_skipped += rows_to_skip
            print(f"Skipped {rows_to_skip} rows, total skipped: {rows_skipped}")
        print(f"Migrating {len(rows)} rows...")
        placeholders = ", ".join(["%s"] * len(columns))
        columns_str = ", ".join(columns)
        insert_query = f"INSERT INTO {target_table_name} ({columns_str}) VALUES ({placeholders})"
        try:
            pg_cursor.executemany(insert_query, rows)
            pg_conn.commit()
        except Exception as e:
            print(f"Error inserting data into PostgreSQL: {e}")
            pg_conn.rollback()
    mysql_cursor.close()
    pg_cursor.close()
    print(f"Migration complete for table {table_name}. Total rows processed: {total_rows_processed}")

def main():
    mysql_conn = connect_mysql(mySQL_connection_params)
    pg_conn = connect_postgresql(target_connection_params)
    tables = ['jobmatl_2']
    for table_name in tables:
        migrate_data(mysql_conn, pg_conn, table_name)
    threading.Thread(target=lambda: time.sleep(300)).start()

mySQL_connection_params = {
    'host': 'MYSQL_HOST',
    'port': '3306',
    'database': 'MYSQL_DATABASE',
    'username': 'MYSQL_USER',
    'password': 'MYSQL_PASSWORD',
}

target_connection_params = {
    'host': 'PG_HOST',
    'port': '5432',
    'database': 'PG_DATABASE',
    'username': 'PG_USER',
    'password': 'PG_PASSWORD',
}

if __name__ == "__main__":
    main()


In [ ]:
#migrating data from MYSQl to postgresql genetrating unique column removing unwanted columns

In [ ]:
Objective
The script migrates data from a MySQL table to a PostgreSQL table, ensuring:

Batch processing for efficiency.

Column exclusions to remove unnecessary data.

Unique primary key generation.

BLOB data conversion for PostgreSQL compatibility.

Handling NULL values properly.

Key Steps in the Script
1. Database Connection
Connects to MySQL using mysql.connector.

Connects to PostgreSQL using psycopg2.

2. Fetching Table Details
Counts total rows in the MySQL table.

Checks already migrated rows in PostgreSQL.

3. Batch-wise Data Migration
Fetches a batch (BATCH_SIZE = 10,000) from MySQL using LIMIT and OFFSET.

Loads the data into a Pandas DataFrame for processing.

Drops unwanted columns (COLUMNS_TO_REMOVE).

Converts BLOB (binary data) into a readable format.

Generates a unique primary key column (aj_pk_column).

Saves the DataFrame to an in-memory CSV (io.StringIO()).

4. Data Insertion into PostgreSQL
Uses PostgreSQL’s COPY command for fast insertion.

Ensures NULL values are handled properly (NULL '\\N').

5. Error Handling & Logging
Tracks time taken for migration.

Commits the batch upon success; rolls back on failure.

Closes all database connections at the end.

Optimization Techniques Used
Batch processing – Reduces memory usage and speeds up migration.

COPY instead of INSERT – Faster data loading in PostgreSQL.

In-memory CSV processing – Avoids unnecessary file I/O.

Skipping already migrated rows – Prevents duplication.


In [ ]:
import pandas as pd
import mysql.connector
import psycopg2
import io  # For in-memory CSV file
import time  # For tracking time
import os  # For environment variables

# MySQL connection details
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT", "3306")
MYSQL_DB = os.getenv("MYSQL_DB")

# PostgreSQL connection details
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT", "5432")
PG_DB = os.getenv("PG_DB")
PG_SCHEMA = os.getenv("PG_SCHEMA")

# Batch size
BATCH_SIZE = 10000

# Columns to exclude from migration
COLUMNS_TO_REMOVE = ["PURGE_SEQ_ID", "PURGE_DATE", "AJ_RETENTION_APPLIED"]

# Table name to migrate
table_name = "ean0000"

try:
    start_time = time.time()  # Start timing
    print(f"Starting migration for table '{table_name}'...")

    # Connect to MySQL
    mysql_conn = mysql.connector.connect(
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        host=MYSQL_HOST,
        port=MYSQL_PORT,
        database=MYSQL_DB
    )
    mysql_cursor = mysql_conn.cursor()

    # Connect to PostgreSQL
    pg_conn = psycopg2.connect(
        user=PG_USER,
        password=PG_PASSWORD,
        host=PG_HOST,
        port=PG_PORT,
        database=PG_DB
    )
    pg_cursor = pg_conn.cursor()

    # Get the total row count from MySQL
    mysql_cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    total_count = mysql_cursor.fetchone()[0]
    print(f"Total rows in MySQL '{table_name}': {total_count}")

    # Determine rows already migrated in PostgreSQL
    pg_cursor.execute(f"SELECT COUNT(*) FROM {PG_SCHEMA}.{table_name}")
    migrated_count = pg_cursor.fetchone()[0]
    print(f"Already migrated rows in PostgreSQL '{table_name}': {migrated_count}")

    if migrated_count >= total_count:
        print(f"Table '{table_name}' already completed. Skipping migration.")
    else:
        print(f"Migrating rows {migrated_count+1} to {total_count} in batches of {BATCH_SIZE}...")
        while migrated_count < total_count:
            query = f"SELECT * FROM {table_name} LIMIT {BATCH_SIZE} OFFSET {migrated_count}"
            mysql_cursor.execute(query)
            data = mysql_cursor.fetchall()

            if not data:
                break

            column_names = [desc[0] for desc in mysql_cursor.description]
            df = pd.DataFrame(data, columns=column_names)
            df = df.drop(columns=COLUMNS_TO_REMOVE, errors='ignore')

            # Convert BLOB columns to hex for PostgreSQL compatibility
            for col in df.select_dtypes(include=['object']).columns:
                if df[col].apply(lambda x: isinstance(x, bytes)).any():
                    df[col] = df[col].apply(lambda x: x.hex() if isinstance(x, bytes) else x)

            df['aj_pk_column'] = range(migrated_count + 1, migrated_count + 1 + len(df))

            # Quote columns with spaces or special characters for PostgreSQL
            columns = ", ".join([f'"{col}"' if ' ' in col or not col.isidentifier() else col for col in df.columns])
            csv_buffer = io.StringIO()
            df.to_csv(csv_buffer, index=False, header=False, na_rep="\\N")
            csv_buffer.seek(0)

            copy_sql = f"COPY {PG_SCHEMA}.{table_name} ({columns}) FROM STDIN WITH CSV NULL '\\N'"
            try:
                pg_cursor.copy_expert(copy_sql, csv_buffer)
                pg_conn.commit()
            except Exception as e:
                pg_conn.rollback()
                print(f"Error during batch commit: {e}")

            migrated_count += df.shape[0]
            print(f"Migrated batch: {df.shape[0]} rows. Total migrated so far: {migrated_count}/{total_count}")

        print(f"Migration completed successfully for '{table_name}'!")

    end_time = time.time()
    time_taken = end_time - start_time
    print(f"Time taken for table '{table_name}': {time_taken:.2f} seconds")

    mysql_cursor.close()
    mysql_conn.close()
    pg_cursor.close()
    pg_conn.close()
    print(f"Database connections closed for table '{table_name}'.")

except Exception as e:
    print(f"Error during migration: {e}")

finally:
    print("Migration script finished.")